In [56]:
# Enabling reloading extension to use external python files
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import columns
import datasets
import seasons
import directory_utils

In [59]:
_MIN_DAY_VALUE = 1
_MAX_DAY_VALUE = 366
_DAYS_RANGE = range(_MIN_DAY_VALUE - 1, _MAX_DAY_VALUE + 1)
print("Days range: " + str(_DAYS_RANGE))

_MIN_HOUR_VALUE = 0
_MAX_HOUR_VALUE = 2400
_HOURS_RANGE = range(_MIN_HOUR_VALUE - 1, _MAX_HOUR_VALUE + 1)
print("Hours range: " + str(_HOURS_RANGE))

_MIN_YEAR_VALUE = 1997
_MAX_YEAR_VALUE = 2016
_YEARS_RANGE = range(_MIN_YEAR_VALUE - 1, _MAX_YEAR_VALUE + 1)
print("Years range: " + str(_YEARS_RANGE))

Days range: range(0, 367)
Hours range: range(-1, 2401)
Years range: range(1996, 2017)


In [62]:
data_frame = pd.read_csv(datasets.MERGED_CSVS__DATA_SET_111_PATH, encoding=datasets.DEFAULT_CSV_ENCODING)

In [63]:
original_shape = data_frame.shape
original_shape

(997961, 18)

In [64]:
data_frame.head(2)

,Cod,Ano,Dia Juliano,Hora - minuto,Veloc. Instantânea do Vento a 5 metros (m/s),Veloc. Instantânea do Vento a 2 metros (m/s),Direção do Vento no instante da aquisição (°),Radiação Incidente total de 10 leituras (KW/m2),Radiação Refletida Total de 10 leituras (KW/m2),Fluxo Positivo de calor no solo total de 10 leituras (W/m2),Fluxo negativo de calor no solo . total de 10 leituras (W/m2),Umidade relativa Média (%),Temperatura do Ar Média (°C),Temperatura do solo 1 Média (°C) à 3cm no solo,Temperatura do solo 2Média (°C) à 6cm no solo,Temperatura do solo 3Média (°C) Colocado sensor de UV em 31/07/09,Pressão (hPa),Chuva total do período (mm)
0,111,1998,1.0,10,2.56,1728.0,129.2,0.0,0.0,0.0,-486.6,100.0,22.14,26.06,27.25,29.02,946.0,0
1,111,1998,1.0,20,2403.00,1682.0,90.0,0.0,0.0,0.0,-482.9,100.0,22.13,26.05,27.25,29.02,946.0,0


In [65]:
data_frame.describe()

,Cod,Ano,Dia Juliano,Hora - minuto,Veloc. Instantânea do Vento a 5 metros (m/s),Veloc. Instantânea do Vento a 2 metros (m/s),Direção do Vento no instante da aquisição (°),Radiação Incidente total de 10 leituras (KW/m2),Radiação Refletida Total de 10 leituras (KW/m2),Fluxo Positivo de calor no solo total de 10 leituras (W/m2),Fluxo negativo de calor no solo . total de 10 leituras (W/m2),Umidade relativa Média (%),Temperatura do Ar Média (°C),Temperatura do solo 1 Média (°C) à 3cm no solo,Temperatura do solo 2Média (°C) à 6cm no solo,Pressão (hPa)
count,997961.000000,997961.000000,997961.000000,997961.000000,997961.000000,997961.000000,997961.000000,997961.000000,997961.000000,997961.000000,996136.000000,996135.000000,996135.000000,995770.000000,995770.000000,995770.000000
mean,111.502900,2007.051017,183.586042,1192.166297,1475.224883,917.671706,239.113364,741.601105,274.279981,370.378574,-1738.391017,73.584838,26.199319,1400.862837,1363.319500,946.107381
std,10.151583,5.468526,105.384332,692.755570,1357.186893,1144.058003,554.738891,1577.788541,904.804691,1171.193961,2848.168698,27.801138,242.126065,2778.728437,2748.226063,82.974140
min,111.000000,1998.000000,-53.360000,10.000000,-6999.000000,0.000000,0.000000,-6999.000000,0.000000,0.000000,-6999.000000,-6999.000000,-6999.000000,-6999.000000,-6999.000000,-6999.000000
25%,111.000000,2002.000000,93.000000,610.000000,1.010000,0.539000,97.700000,0.000000,0.000000,0.000000,-639.400000,60.150000,18.180000,20.530000,21.210000,942.000000
50%,111.000000,2007.000000,184.000000,1210.000000,1481.000000,1.010000,156.800000,0.000000,0.000000,0.000000,-246.200000,77.100000,21.180000,24.520000,24.970000,947.000000
75%,111.000000,2012.000000,275.000000,1810.000000,2265.000000,1716.000000,256.500000,426.200000,1.180000,166.600000,0.000000,88.600000,25.140000,32.240000,30.000000,952.000000
max,333.000000,2139.000000,654.000000,6640.000000,6996.000000,6999.000000,6999.000000,6999.000000,6999.000000,6999.000000,6538.000000,6836.000000,182266.000000,6999.000000,6999.000000,2802.000000


### Removing invalid days, hours and years

In [66]:
def is_in_range(range, value):
    return value in range
def is_not_in_range(range, value):
    return value not in range

In [69]:
_days_not_in_range_filter = data_frame[columns.JULIAN_DAY].map(lambda value: is_not_in_range(_DAYS_RANGE, value))
data_frame[_days_not_in_range_filter][columns.JULIAN_DAY].unique()

array([-53.36, 388.  ,  29.77, 654.  ])

In [70]:
_days_in_range_filter = data_frame[columns.JULIAN_DAY].map(lambda value: is_in_range(_DAYS_RANGE, value))
data_frame = data_frame[_days_in_range_filter]

In [71]:
_years_not_in_range_filter = data_frame[columns.YEAR].map(lambda value: is_not_in_range(_YEARS_RANGE, value))
data_frame[_years_not_in_range_filter][columns.YEAR].unique()

array([2017])

In [73]:
_years_in_range_filter = data_frame[columns.YEAR].map(lambda value: is_in_range(_YEARS_RANGE, value))
data_frame = data_frame[_years_in_range_filter]

In [74]:
_hours_not_in_range_filter = data_frame[columns.HOUR_MINUTE].map(lambda value: is_not_in_range(_HOURS_RANGE, value))

data_frame[_hours_not_in_range_filter][columns.HOUR_MINUTE].unique()

array([], dtype=int64)

In [75]:
_hours_in_range_filter = data_frame[columns.HOUR_MINUTE].map(lambda value: is_in_range(_HOURS_RANGE, value))
data_frame = data_frame[_hours_in_range_filter]

In [76]:
data_frame.shape

(997785, 18)

### Removing null values

In [77]:
data_frame.isnull().sum()

Cod                                                                     0
Ano                                                                     0
Dia Juliano                                                             0
Hora - minuto                                                           0
Veloc. Instantânea do Vento a 5 metros (m/s)                            0
Veloc. Instantânea do Vento a 2 metros (m/s)                            0
Direção do Vento no instante da aquisição (°)                           0
Radiação Incidente total de 10 leituras (KW/m2)                         0
Radiação Refletida Total de 10 leituras (KW/m2)                         0
Fluxo Positivo de calor no solo total de 10 leituras (W/m2)             0
Fluxo negativo de calor no solo . total de 10 leituras (W/m2)        1825
Umidade relativa Média (%)                                           1826
Temperatura do Ar Média (°C)                                         1826
Temperatura do solo  1 Média (°C) à 3c

In [78]:
data_frame.shape

(997785, 18)

In [79]:
data_frame.dropna().shape

(995594, 18)

In [80]:
data_frame = data_frame.dropna()

In [81]:
data_frame.isnull().sum()

Cod                                                                  0
Ano                                                                  0
Dia Juliano                                                          0
Hora - minuto                                                        0
Veloc. Instantânea do Vento a 5 metros (m/s)                         0
Veloc. Instantânea do Vento a 2 metros (m/s)                         0
Direção do Vento no instante da aquisição (°)                        0
Radiação Incidente total de 10 leituras (KW/m2)                      0
Radiação Refletida Total de 10 leituras (KW/m2)                      0
Fluxo Positivo de calor no solo total de 10 leituras (W/m2)          0
Fluxo negativo de calor no solo . total de 10 leituras (W/m2)        0
Umidade relativa Média (%)                                           0
Temperatura do Ar Média (°C)                                         0
Temperatura do solo  1 Média (°C) à 3cm no solo                      0
Temper

In [82]:
data_frame.shape

(995594, 18)

### Removing days with value "0"

In [84]:
data_frame[data_frame[columns.JULIAN_DAY]==0]

,Cod,Ano,Dia Juliano,Hora - minuto,Veloc. Instantânea do Vento a 5 metros (m/s),Veloc. Instantânea do Vento a 2 metros (m/s),Direção do Vento no instante da aquisição (°),Radiação Incidente total de 10 leituras (KW/m2),Radiação Refletida Total de 10 leituras (KW/m2),Fluxo Positivo de calor no solo total de 10 leituras (W/m2),Fluxo negativo de calor no solo . total de 10 leituras (W/m2),Umidade relativa Média (%),Temperatura do Ar Média (°C),Temperatura do solo 1 Média (°C) à 3cm no solo,Temperatura do solo 2Média (°C) à 6cm no solo,Temperatura do solo 3Média (°C) Colocado sensor de UV em 31/07/09,Pressão (hPa),Chuva total do período (mm)
661589,111,2000,0.0,10,2187.0,2187.0,354.5,0.0,0.0,42.23,0.0,35.45,27.75,20.49,18.88,4.4,949.0,0
661590,111,2000,0.0,20,1952.0,1952.0,319.8,0.0,0.0,441.90,0.0,36.56,27.51,20.80,18.97,4497,949.0,0
661591,111,2000,0.0,30,2187.0,2187.0,95.6,0.0,0.0,471.80,0.0,36.23,27.65,21.05,19.09,4506,949.0,0
661592,111,2000,0.0,40,3128.0,2893.0,79.1,0.0,0.0,499.40,0.0,34.85,28.19,21.35,19.16,4717,948.0,0


In [85]:
data_frame = data_frame[data_frame[columns.JULIAN_DAY]!=0]

In [86]:
data_frame.shape

(995590, 18)

### Removing data from 222

In [88]:
data_frame[columns.CODE].unique()

array([111, 222])

In [89]:
data_frame[data_frame[columns.CODE]!=111].head()

,Cod,Ano,Dia Juliano,Hora - minuto,Veloc. Instantânea do Vento a 5 metros (m/s),Veloc. Instantânea do Vento a 2 metros (m/s),Direção do Vento no instante da aquisição (°),Radiação Incidente total de 10 leituras (KW/m2),Radiação Refletida Total de 10 leituras (KW/m2),Fluxo Positivo de calor no solo total de 10 leituras (W/m2),Fluxo negativo de calor no solo . total de 10 leituras (W/m2),Umidade relativa Média (%),Temperatura do Ar Média (°C),Temperatura do solo 1 Média (°C) à 3cm no solo,Temperatura do solo 2Média (°C) à 6cm no solo,Temperatura do solo 3Média (°C) Colocado sensor de UV em 31/07/09,Pressão (hPa),Chuva total do período (mm)
363935,222,2005,1.0,2400,4894.00,4344.00,113.9,1805.0,158.4,78.10,25.33,-18.58,32.85,1321.0,20.07,529,33.97,1256
364086,222,2005,2.0,2400,6.07,5.60,140.8,1517.0,131.5,70.70,20.15,-22.71,32.06,1259.0,19.55,524,34.08,1314
364237,222,2005,3.0,2400,5168.00,4266.00,270.6,1637.0,136.5,74.30,20.47,-22.35,31.32,1314.0,18.11,534,32.32,1326
364388,222,2005,4.0,2400,4462.00,3717.00,302.0,2216.0,75.5,51.65,5698.00,-19.32,24.66,1537.0,18.47,200,27.85,1550
364539,222,2005,5.0,2400,4423.00,4.07,358.2,1514.0,101.0,60.42,9.87,-13.45,26.20,1441.0,18.60,335,27.78,1650


In [91]:
data_frame = data_frame[data_frame[columns.CODE]==111]

In [92]:
data_frame.shape

(995225, 18)

### Removing invalid Air Temperature measurements

In [95]:
_MIN_AIR_TEMPERATURE=-2
_MAX_AIR_TEMPERATURE=45

In [96]:
data_frame[columns.TEMPERATURE].describe()

count    995225.000000
mean         25.678895
std         240.480127
min       -6999.000000
25%          18.170000
50%          21.180000
75%          25.130000
max      182266.000000
Name: Temperatura do Ar Média (°C), dtype: float64

In [97]:
data_frame_without_invalid_air_temp = data_frame.loc[(data_frame[columns.TEMPERATURE] > _MIN_AIR_TEMPERATURE) & (data_frame[columns.TEMPERATURE] < _MAX_AIR_TEMPERATURE), :]
data_frame_without_invalid_air_temp[columns.TEMPERATURE].describe()

count    994436.000000
mean         21.620166
std           5.043158
min           3.890000
25%          18.170000
50%          21.180000
75%          25.120000
max          44.990000
Name: Temperatura do Ar Média (°C), dtype: float64

In [98]:
data_frame.shape[0] - data_frame_without_invalid_air_temp.shape[0]

789

In [99]:
data_frame = data_frame_without_invalid_air_temp

### Removing invalid Air moisture measurements

In [100]:
_MIN_AIR_MOISTURE=0
_MAX_AIR_MOISTURE=100

In [101]:
data_frame[columns.HUMIDITY].describe()

count    994436.000000
mean         73.626116
std          26.046411
min        -237.800000
25%          60.170000
50%          77.100000
75%          88.600000
max        6836.000000
Name: Umidade relativa Média (%), dtype: float64

In [102]:
data_frame_without_invalid_air_mois = data_frame.loc[(data_frame[columns.HUMIDITY] >= _MIN_AIR_MOISTURE) & (data_frame[columns.HUMIDITY] <= _MAX_AIR_MOISTURE), :]
data_frame_without_invalid_air_mois[columns.HUMIDITY].describe()

count    994424.000000
mean         73.570277
std          18.713490
min           7.410000
25%          60.170000
50%          77.100000
75%          88.600000
max         100.000000
Name: Umidade relativa Média (%), dtype: float64

In [103]:
data_frame.shape[0] - data_frame_without_invalid_air_mois.shape[0]

12

In [104]:
data_frame = data_frame_without_invalid_air_mois

### Removing invalid Air pressure measurements

In [108]:
_MIN_AIR_PRESSURE=822
_MAX_AIR_PRESSURE=1030

In [109]:
data_frame[columns.PRESSURE].describe()

count    994424.000000
mean        946.432905
std          80.712954
min       -6999.000000
25%         942.000000
50%         947.000000
75%         952.000000
max        2802.000000
Name: Pressão (hPa), dtype: float64

In [110]:
data_frame_without_invalid_air_press = data_frame.loc[(data_frame[columns.PRESSURE] >= _MIN_AIR_PRESSURE) & (data_frame[columns.PRESSURE] <= _MAX_AIR_PRESSURE), :]
data_frame_without_invalid_air_press[columns.PRESSURE].describe()

count    994234.000000
mean        947.238158
std           6.734704
min         822.000000
25%         942.000000
50%         947.000000
75%         952.000000
max        1028.000000
Name: Pressão (hPa), dtype: float64

In [111]:
data_frame.shape[0] - data_frame_without_invalid_air_press.shape[0]

190

In [112]:
data_frame = data_frame_without_invalid_air_press

### Removing invalid Wind Directionmeasurements

In [113]:
_MIN_WIND_DIRECTION=0
_MAX_WIND_DIRECTION=360

In [114]:
data_frame[columns.WIND_DIRECTION].describe()

count    994234.000000
mean        238.645789
std         554.298967
min           0.000000
25%          97.600000
50%         156.800000
75%         256.500000
max        6999.000000
Name: Direção do Vento no instante da aquisição (°), dtype: float64

In [115]:
data_frame_without_invalid_wind_dir = data_frame.loc[(data_frame[columns.WIND_DIRECTION] >= _MIN_WIND_DIRECTION) & (data_frame[columns.WIND_DIRECTION] <= _MAX_WIND_DIRECTION), :]
data_frame_without_invalid_wind_dir[columns.WIND_DIRECTION].describe()

count    973821.000000
mean        168.790247
std          96.835213
min           0.000000
25%          95.800000
50%         154.200000
75%         250.900000
max         360.000000
Name: Direção do Vento no instante da aquisição (°), dtype: float64

In [116]:
data_frame.shape[0] - data_frame_without_invalid_wind_dir.shape[0]

20413

In [117]:
# I'm commenting this for now to avoid removing valid temperature and pression measurements.
#data_frame = data_frame_without_invalid_wind_dir

### Results after cleaning

In [119]:
original_amount_of_lines = original_shape[0]
amount_of_lines_after_cleaning = data_frame.shape[0]
amount_of_lines_removed = original_amount_of_lines - amount_of_lines_after_cleaning

percentage_of_lines_removed = 100 * amount_of_lines_removed / original_amount_of_lines

percentage_of_lines_removed = round(percentage_of_lines_removed, 2)

print(f"{amount_of_lines_removed} observations were removed what is {percentage_of_lines_removed}% of the dataset")


3727 observations were removed what is 0.37% of the dataset


### Creating the date column

In [120]:
year_mapping = data_frame[columns.YEAR].map(lambda year : datetime(year, 1, 1))
time_delta_mapping = data_frame[columns.JULIAN_DAY].map(lambda day : timedelta(day-1))
data_frame[columns.DATE] = year_mapping + time_delta_mapping

/home/felipepedroso/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Saving CSV

In [121]:
data_frame.to_csv(datasets.FIRST_CLEAN_DATA_SET_111_PATH, index = False, encoding=datasets.DEFAULT_CSV_ENCODING)